In [53]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.externals import joblib
from sklearn.grid_search import GridSearchCV
from xgboost import XGBClassifier

from DNN_preprocessing import getData

%matplotlib inline

元データをロード

In [2]:
df = getData(fillnan=False, savedata=False)
df.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,EMERGENCYSTATE_MODE,label,IS_TEST
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,-2120,...,0,0,0,0,1.0,0,1,0.0,1.0,False
100003,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,-291,...,0,0,0,0,0.0,0,0,0.0,0.0,False
100004,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,-2531,...,0,0,0,1,1.0,1,1,NaN,0.0,False
100006,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,-2437,...,0,0,0,0,0.0,0,1,NaN,0.0,False
100007,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,-3458,...,0,0,0,0,1.0,0,1,NaN,0.0,False


訓練データとテストデータを分割

In [13]:
df_train = df[-df['IS_TEST']].drop('IS_TEST', axis=1)
df_test = df[df['IS_TEST']].drop('IS_TEST', axis=1)

trX, trY = df_train.drop('label', axis=1), df_train['label']
valX = df_test.drop('label', axis=1)

#### XGBoostのGrid Search  
とりあえずパラメータチューニングなしの結果を出力

In [17]:
xgb=XGBClassifier()
xgb.fit(trX, trY)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [19]:
preds = xgb.predict_proba(valX)[:,1]
sub_lgb = pd.DataFrame()
sub_lgb['SK_ID_CURR'] = df_test.index.tolist()
sub_lgb['TARGET'] = preds
sub_lgb.to_csv("xgb_non_gridsearch.csv", index=False)
sub_lgb.head()

,SK_ID_CURR,TARGET
0,100001,0.084083
1,100005,0.171885
2,100013,0.025795
3,100028,0.038975
4,100038,0.140532


これでAUC=0.737
#### Grid Search  
以下urlを参考にGrid Searchをやってみる  
http://kamonohashiperry.com/archives/500  

##### max_depthとmin_child_weightのチューニング

In [25]:
param_test1 = {
    'max_depth':list(range(3,10,2)),
    'min_child_weight':list(range(1,6,2))
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=100, max_depth=5,
                                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=-1,iid=False, cv=3)
gsearch1.fit(trX,trY)

GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=100, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=-1,
       param_grid={'max_depth': [3, 5, 7, 9], 'min_child_weight': [1, 3, 5]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [27]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_ 

([mean: 0.74928, std: 0.00218, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.74910, std: 0.00221, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.74894, std: 0.00200, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.75407, std: 0.00125, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.75403, std: 0.00157, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.75434, std: 0.00170, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.75423, std: 0.00087, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.75473, std: 0.00195, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.75430, std: 0.00187, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.75111, std: 0.00109, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.75159, std: 0.00155, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.75134, std: 0.00158, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 7, 'min_child_weight': 3

In [29]:
preds = gsearch1.best_estimator_.predict_proba(valX)[:,1]
sub_lgb = pd.DataFrame()
sub_lgb['SK_ID_CURR'] = df_test.index.tolist()
sub_lgb['TARGET'] = preds
sub_lgb.to_csv("xgb_gridsearch1.csv", index=False)
sub_lgb.head()

,SK_ID_CURR,TARGET
0,100001,0.045730
1,100005,0.186475
2,100013,0.021593
3,100028,0.043985
4,100038,0.149575


これで0.742

より細かい数値でチューニング

In [30]:
param_test2 = {
    'max_depth':[6,7,8],
    'min_child_weight':[2,3,4]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=100, max_depth=5,
                                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                       param_grid = param_test2, scoring='roc_auc',n_jobs=-1,iid=False, cv=3)
gsearch2.fit(trX,trY)

GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=100, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=-1,
       param_grid={'max_depth': [6, 7, 8], 'min_child_weight': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [31]:
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_ 

([mean: 0.75442, std: 0.00158, params: {'max_depth': 6, 'min_child_weight': 2},
  mean: 0.75500, std: 0.00173, params: {'max_depth': 6, 'min_child_weight': 3},
  mean: 0.75445, std: 0.00173, params: {'max_depth': 6, 'min_child_weight': 4},
  mean: 0.75380, std: 0.00168, params: {'max_depth': 7, 'min_child_weight': 2},
  mean: 0.75473, std: 0.00195, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.75428, std: 0.00157, params: {'max_depth': 7, 'min_child_weight': 4},
  mean: 0.75289, std: 0.00144, params: {'max_depth': 8, 'min_child_weight': 2},
  mean: 0.75277, std: 0.00154, params: {'max_depth': 8, 'min_child_weight': 3},
  mean: 0.75330, std: 0.00136, params: {'max_depth': 8, 'min_child_weight': 4}],
 {'max_depth': 6, 'min_child_weight': 3},
 0.7549967980980807)

In [32]:
preds = gsearch2.best_estimator_.predict_proba(valX)[:,1]
sub_lgb = pd.DataFrame()
sub_lgb['SK_ID_CURR'] = df_test.index.tolist()
sub_lgb['TARGET'] = preds
sub_lgb.to_csv("xgb_gridsearch2.csv", index=False)
sub_lgb.head()

,SK_ID_CURR,TARGET
0,100001,0.059971
1,100005,0.179630
2,100013,0.039053
3,100028,0.049073
4,100038,0.130165


これで0.746

gammaのチューニング

In [36]:
param_test3 = {
    'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=100, max_depth=6,
                                        min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                       param_grid = param_test3, scoring='roc_auc',n_jobs=-1,iid=False, cv=3)
gsearch3.fit(trX,trY)

GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=3, missing=None, n_estimators=100, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=-1,
       param_grid={'gamma': [0.0, 0.1, 0.2, 0.3, 0.4]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [37]:
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_ 

([mean: 0.75500, std: 0.00173, params: {'gamma': 0.0},
  mean: 0.75472, std: 0.00162, params: {'gamma': 0.1},
  mean: 0.75447, std: 0.00140, params: {'gamma': 0.2},
  mean: 0.75452, std: 0.00140, params: {'gamma': 0.3},
  mean: 0.75484, std: 0.00159, params: {'gamma': 0.4}],
 {'gamma': 0.0},
 0.7549967980980807)

In [38]:
preds = gsearch3.best_estimator_.predict_proba(valX)[:,1]
sub_lgb = pd.DataFrame()
sub_lgb['SK_ID_CURR'] = df_test.index.tolist()
sub_lgb['TARGET'] = preds
sub_lgb.to_csv("xgb_gridsearch3.csv", index=False)
sub_lgb.head()

,SK_ID_CURR,TARGET
0,100001,0.059971
1,100005,0.179630
2,100013,0.039053
3,100028,0.049073
4,100038,0.130165


これで0.746

subsampleとcolsample_bytreeのチューニング

In [39]:
param_test4 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=100, max_depth=6,
                                        min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                        param_grid = param_test4, scoring='roc_auc',n_jobs=-1,iid=False, cv=3)
gsearch4.fit(trX,trY)

GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=3, missing=None, n_estimators=100, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=-1,
       param_grid={'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [40]:
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_ 

([mean: 0.75395, std: 0.00185, params: {'subsample': 0.6, 'colsample_bytree': 0.6},
  mean: 0.75457, std: 0.00123, params: {'subsample': 0.7, 'colsample_bytree': 0.6},
  mean: 0.75453, std: 0.00136, params: {'subsample': 0.8, 'colsample_bytree': 0.6},
  mean: 0.75493, std: 0.00090, params: {'subsample': 0.9, 'colsample_bytree': 0.6},
  mean: 0.75341, std: 0.00125, params: {'subsample': 0.6, 'colsample_bytree': 0.7},
  mean: 0.75476, std: 0.00111, params: {'subsample': 0.7, 'colsample_bytree': 0.7},
  mean: 0.75476, std: 0.00172, params: {'subsample': 0.8, 'colsample_bytree': 0.7},
  mean: 0.75476, std: 0.00156, params: {'subsample': 0.9, 'colsample_bytree': 0.7},
  mean: 0.75403, std: 0.00185, params: {'subsample': 0.6, 'colsample_bytree': 0.8},
  mean: 0.75540, std: 0.00152, params: {'subsample': 0.7, 'colsample_bytree': 0.8},
  mean: 0.75500, std: 0.00173, params: {'subsample': 0.8, 'colsample_bytree': 0.8},
  mean: 0.75488, std: 0.00118, params: {'subsample': 0.9, 'colsample_bytree'

細かい範囲で調整

In [41]:
param_test5 = {
    'subsample':[i/100.0 for i in range(65,80,5)],
    'colsample_bytree':[i/100.0 for i in range(75,90,5)]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=100, max_depth=6,
                                        min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                        param_grid = param_test5, scoring='roc_auc',n_jobs=-1,iid=False, cv=3)
gsearch5.fit(trX,trY)

GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=3, missing=None, n_estimators=100, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8),
       fit_params={}, iid=False, n_jobs=-1,
       param_grid={'subsample': [0.65, 0.7, 0.75], 'colsample_bytree': [0.75, 0.8, 0.85]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [42]:
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_ 

([mean: 0.75388, std: 0.00186, params: {'subsample': 0.65, 'colsample_bytree': 0.75},
  mean: 0.75402, std: 0.00191, params: {'subsample': 0.7, 'colsample_bytree': 0.75},
  mean: 0.75496, std: 0.00176, params: {'subsample': 0.75, 'colsample_bytree': 0.75},
  mean: 0.75436, std: 0.00116, params: {'subsample': 0.65, 'colsample_bytree': 0.8},
  mean: 0.75540, std: 0.00152, params: {'subsample': 0.7, 'colsample_bytree': 0.8},
  mean: 0.75486, std: 0.00148, params: {'subsample': 0.75, 'colsample_bytree': 0.8},
  mean: 0.75427, std: 0.00116, params: {'subsample': 0.65, 'colsample_bytree': 0.85},
  mean: 0.75485, std: 0.00173, params: {'subsample': 0.7, 'colsample_bytree': 0.85},
  mean: 0.75443, std: 0.00114, params: {'subsample': 0.75, 'colsample_bytree': 0.85}],
 {'colsample_bytree': 0.8, 'subsample': 0.7},
 0.7554041643184152)

reg_alphaのチューニング

In [45]:
param_test6 = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=100, max_depth=6,
                                        min_child_weight=3, gamma=0, subsample=0.7, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                        param_grid = param_test6, scoring='roc_auc',n_jobs=-1,iid=False, cv=3)
gsearch6.fit(trX,trY)

GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=3, missing=None, n_estimators=100, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.7),
       fit_params={}, iid=False, n_jobs=-1,
       param_grid={'reg_alpha': [1e-05, 0.01, 0.1, 1, 100]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [46]:
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_ 

([mean: 0.75540, std: 0.00152, params: {'reg_alpha': 1e-05},
  mean: 0.75473, std: 0.00153, params: {'reg_alpha': 0.01},
  mean: 0.75445, std: 0.00131, params: {'reg_alpha': 0.1},
  mean: 0.75486, std: 0.00151, params: {'reg_alpha': 1},
  mean: 0.75054, std: 0.00224, params: {'reg_alpha': 100}],
 {'reg_alpha': 1e-05},
 0.7554041660283803)

細かい範囲で調整

In [47]:
param_test7 = {
    'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
}
gsearch7 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=100, max_depth=6,
                                        min_child_weight=3, gamma=0, subsample=0.7, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                        param_grid = param_test7, scoring='roc_auc',n_jobs=-1,iid=False, cv=3)
gsearch7.fit(trX,trY)

GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=3, missing=None, n_estimators=100, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.7),
       fit_params={}, iid=False, n_jobs=-1,
       param_grid={'reg_alpha': [0, 0.001, 0.005, 0.01, 0.05]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [48]:
gsearch7.grid_scores_, gsearch7.best_params_, gsearch7.best_score_ 

([mean: 0.75540, std: 0.00152, params: {'reg_alpha': 0},
  mean: 0.75527, std: 0.00160, params: {'reg_alpha': 0.001},
  mean: 0.75450, std: 0.00183, params: {'reg_alpha': 0.005},
  mean: 0.75473, std: 0.00153, params: {'reg_alpha': 0.01},
  mean: 0.75421, std: 0.00178, params: {'reg_alpha': 0.05}],
 {'reg_alpha': 0},
 0.7554041643184152)

n_estimatorsを増やして最終的なモデルを推定

In [57]:
xgb_final = XGBClassifier(learning_rate =0.1, 
                          n_estimators=100, 
                          max_depth=6, 
                          min_child_weight=3, 
                          gamma=0, 
                          subsample=0.7, 
                          colsample_bytree=0.8,
                          reg_alpha=0, 
                          objective= 'binary:logistic', 
                          nthread=4, 
                          scale_pos_weight=1, 
                          seed=27)
xgb_final.fit(trX,trY)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=3, missing=None, n_estimators=100, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.7)

In [58]:
preds = xgb_final.predict_proba(valX)[:,1]
sub_lgb = pd.DataFrame()
sub_lgb['SK_ID_CURR'] = df_test.index.tolist()
sub_lgb['TARGET'] = preds
sub_lgb.to_csv("xgb_gridsearch_final3.csv", index=False)
sub_lgb.head()

,SK_ID_CURR,TARGET
0,100001,0.060336
1,100005,0.181642
2,100013,0.024840
3,100028,0.038187
4,100038,0.149205


モデルを保存しておく

In [54]:
joblib.dump(xgb_final, 'xgb_final.pkl')

['xgb_final.pkl']